In [17]:
input_query = 'RAG'

In [18]:
import psycopg2
import os

def connection():
    """Creates and returns a new database connection."""
    try:
        conn = psycopg2.connect(
            user=os.environ["MY_INTEGRATION_USER"],
            password=os.environ["MY_INTEGRATION_PASSWORD"],
            host=os.environ["MY_INTEGRATION_HOST"],
            port=os.environ["MY_INTEGRATION_PORT"],
            database=os.environ["MY_INTEGRATION_DATABASE"]
        )
        
        # Test the connection
        with conn.cursor() as cursor:
            cursor.execute("SELECT version();")
            record = cursor.fetchone()
        
        return conn  # Return the connection object if successful

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to database", error)
        return None  # Return None if connection was not successful

conn = connection()

In [20]:
import sqlite3

def add_new_job(query):
    # Connect to the SQLite database
    conn = connection()
    c = conn.cursor()

    # SQL statement to insert a new job
    c.execute("INSERT INTO jobs (query, job_status) VALUES (%s, 'new')", (query,))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

# Usage
job_query = "Top academic papers on " + input_query
add_new_job(job_query)

In [24]:
import sqlite3
import json
from rich.console import Console
from rich.table import Table
from rich.text import Text

def display_query_papers(job_query):
    # Create a console object for Rich output
    console = Console()

    # Connect to SQLite database
    conn = connection()
    c = conn.cursor()

    # Fetch all records for the given query sorted by final_rank
    c.execute("""
        SELECT * FROM Query_Papers 
        WHERE query = %s 
        ORDER BY final_rank
        LIMIT 10  -- Limit the results to top 10
    """, (job_query,))


    # Fetch the column names
    columns = [description[0] for description in c.description]

    # Fetch all rows from the query
    rows = c.fetchall()

    # Check if the result set is not empty
    if rows:
        # Initialize a Rich table with improved formatting
        table = Table(show_header=True, title=f"{job_query}", expand=True, leading=1, show_lines=True)

        # Define the columns
        table.add_column("No.", style="cyan", justify="right", ratio=1)
        table.add_column("Paper", overflow="fold", ratio=20)  # This has twice the ratio of "Details", meaning it will be larger
        table.add_column("Details", overflow="fold", ratio=8)  # Half the 'ratio' of "Paper", making it relatively smaller
        table.add_column("Link", justify="center", ratio=2)

        for idx, row in enumerate(rows, start=1):
            # Extract the fields from the row
            arxiv_link = row[columns.index('arxiv_link')]
            relevant_answer = row[columns.index('relevant_answer')]
            # Properly handle 'None' values for 'paper_stats' and 'paper_metadata_filtered'
            paper_stats_str = row[columns.index('paper_stats')] or '{}'
            paper_metadata_filtered_str = row[columns.index('paper_metadata_filtered')] or '{}'

            # Attempt to decode JSON, defaulting to empty dictionary on failure
            try:
                paper_stats = json.loads(paper_stats_str)
            except json.JSONDecodeError:
                paper_stats = {}  # Default to empty dict if there is a JSON decode error
                
            try:
                paper_metadata_filtered = json.loads(paper_metadata_filtered_str)
            except json.JSONDecodeError:
                paper_metadata_filtered = {}  # Default to empty dict if there is a JSON decode error

            # Extract and format data from decoded JSON
            citations = str(paper_stats.get('citations', 'N/A'))
            versions = str(paper_stats.get('versions', 'N/A'))
            title = paper_metadata_filtered.get('title', 'N/A')
            abstract = paper_metadata_filtered.get('abstract', 'N/A')
            if len(abstract) > 200:
                abstract = abstract[:200] + "..."
            published_date = paper_metadata_filtered.get('published_date', 'N/A')
            if published_date != 'N/A':
                published_date = published_date.split('T')[0]  # Keep only the date part
            authors = paper_metadata_filtered.get('authors', ['N/A'])
            authors_str = ", ".join(authors[:3]) + "..." if len(authors) > 3 else ", ".join(authors)

            # Add the clickable 'Link' text
            link_text = f"[link={arxiv_link}]Link[/link]"

            # Format the Paper and Details columns
            paper_column = Text(f"{title}\n\nLLM response: {relevant_answer}\n\nAbstract: {abstract}", justify="left")
            details_column = Text(f"Citations: {citations}\nVersions: {versions}\nDate Published: {published_date}\nAuthors: {authors_str} \n", justify="left")
            
            # Add row with formatted data
            table.add_row(str(idx), paper_column, details_column, link_text)
            table.add_section()
            table.add_row()

        # Print the table to the console
        console.print(table)
    else:
        console.print(f"No records found for query: {job_query}.")

In [25]:
import sqlite3
import time
import datetime  # Import the datetime module
import os  # Import the os module for clearing the terminal
from IPython.display import clear_output

def wait_for_job_completion(job_query):
    # Connect to the SQLite database
    conn = connection()
    c = conn.cursor()
    counter = 0  # Initialize the counter
    try:
        while True:  # Keep checking until the job is done
            # SQL statement to find the status of a job given its query
            c.execute("SELECT job_status FROM jobs WHERE query = %s", (job_query,))
            result = c.fetchone()
            if result:
                job_status = result[0]
                if job_status == 'done':
                    print("\nThe job '{}' is complete.".format(job_query))  # Ensure new line before final status
                    display_query_papers(job_query)
                    break  # Exit the loop if the job is done
                elif job_status == 'running':
                    clear_output(wait=True)  # Clear output and wait for the next
                    counter += 1
                    print(f"{counter}", end=' ')  # Print counter with "Running..."
                    print("\nThe status of the job '{}' is currently '{}'. Waiting for completion...".format(job_query, job_status))
                    display_query_papers(job_query)
                    time.sleep(5)
                else:
                    print("\nThe status of the job '{}' is currently '{}'. Waiting for completion...".format(job_query, job_status))
                    display_query_papers(job_query)  # Indented to match the 'else' block
            else:
                print("\nHello there, we're busy working on your query: '{}'.".format(job_query))
                break  # Exit the loop if no such job exists
    except Exception as e:
        print(f"An error occurred: {e}")

# Usage
wait_for_job_completion(job_query)


29 
The status of the job 'Top academic papers on RAG' is currently 'running'. Waiting for completion...
Error while connecting to database FATAL:  Max client connections reached

An error occurred: 'NoneType' object has no attribute 'cursor'


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6d52007a-f237-4857-b1f1-3ccb95216ee4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>